<a href="https://colab.research.google.com/github/PearlSikka/language-ninja/blob/master/Text_generation_using_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

I will be using the dataset of New York Times Comments and Headlines to train a text generation language model which can be used to generate News Headlines


In [ ]:
import os
import pathlib

# Upload the API token.
def get_kaggle():
  try:
    import kaggle
    return kaggle
  except OSError:
    pass

  token_file = pathlib.Path("~/.kaggle/kaggle.json").expanduser()
  token_file.parent.mkdir(exist_ok=True, parents=True)

  try:
    from google.colab import files
  except ImportError:
    raise ValueError("Could not find kaggle token.")

  uploaded = files.upload()
  token_content = uploaded.get('kaggle.json', None)
  if token_content:
    token_file.write_bytes(token_content)
    token_file.chmod(0o600)
  else:
    raise ValueError('Need a file named "kaggle.json"')
  
  import kaggle
  return kaggle


kaggle = get_kaggle()

Saving kaggle.json to kaggle.json


In [ ]:
!kaggle datasets download -d aashita/nyt-comments

 99% 473M/480M [00:13<00:00, 26.1MB/s]
100% 480M/480M [00:13<00:00, 38.1MB/s]


In [ ]:
!unzip nyt-comments.zip -d train

Archive:  nyt-comments.zip
  inflating: train/ArticlesApril2017.csv  
  inflating: train/ArticlesApril2018.csv  
  inflating: train/ArticlesFeb2017.csv  
  inflating: train/ArticlesFeb2018.csv  
  inflating: train/ArticlesJan2017.csv  
  inflating: train/ArticlesJan2018.csv  
  inflating: train/ArticlesMarch2017.csv  
  inflating: train/ArticlesMarch2018.csv  
  inflating: train/ArticlesMay2017.csv  
  inflating: train/CommentsApril2017.csv  
  inflating: train/CommentsApril2018.csv  
  inflating: train/CommentsFeb2017.csv  
  inflating: train/CommentsFeb2018.csv  
  inflating: train/CommentsJan2017.csv  
  inflating: train/CommentsJan2018.csv  
  inflating: train/CommentsMarch2017.csv  
  inflating: train/CommentsMarch2018.csv  
  inflating: train/CommentsMay2017.csv  


In [ ]:
import pandas as pd
import tensorflow as tf
import numpy as np


from numpy.random import seed

tf.random.set_seed(2)
seed(1)



In [ ]:
import os

curr_dir='/content/train/'

docs=[]
for filename in os.listdir(curr_dir):
  if 'Articles' in filename:
    data=pd.read_csv(curr_dir+filename)
    docs.append(data)
    break
frame=pd.concat(docs,axis=0)

print (frame[:10])


  abstract  ...                                             webURL
0      NaN  ...  https://www.nytimes.com/2018/01/30/travel/braz...
1      NaN  ...  https://www.nytimes.com/2018/01/30/us/politics...
2      NaN  ...  https://www.nytimes.com/2018/01/30/nyregion/me...
3      NaN  ...  https://www.nytimes.com/2018/01/30/opinion/edi...
4      NaN  ...  https://www.nytimes.com/2018/01/30/opinion/chi...
5      NaN  ...  https://www.nytimes.com/2018/01/30/world/afric...
6      NaN  ...  https://www.nytimes.com/2018/01/30/opinion/tru...
7      NaN  ...  https://www.nytimes.com/2018/01/30/sports/larr...
8      NaN  ...  https://www.nytimes.com/2018/01/30/nyregion/de...
9      NaN  ...  https://www.nytimes.com/2018/01/30/books/revie...

[10 rows x 16 columns]


In [ ]:
frame.columns

Index(['abstract', 'articleID', 'articleWordCount', 'byline', 'documentType',
       'headline', 'keywords', 'multimedia', 'newDesk', 'printPage', 'pubDate',
       'sectionName', 'snippet', 'source', 'typeOfMaterial', 'webURL'],
      dtype='object')

In [ ]:
frame.headline[:10]

0    Rhythm of the Streets: ‘We’re Warrior Women, a...
1            As Deficit Grows, Congress Keeps Spending
2                         Lesson in Select Bus Service
3                   Here’s the Real State of the Union
4                         Good Riddance to Chief Wahoo
5     In South Africa, Facing ‘Day Zero’ With No Water
6                   How Trump’s Critics Should Respond
7                                              Unknown
8    A Republican Stalwart Sets Out on a Quest to U...
9                                              Unknown
Name: headline, dtype: object

In [ ]:
frame.shape

(905, 16)

In [ ]:
frame=frame[['headline']]

In [ ]:
frame.head()

,headline
0,"Rhythm of the Streets: ‘We’re Warrior Women, a..."
1,"As Deficit Grows, Congress Keeps Spending"
2,Lesson in Select Bus Service
3,Here’s the Real State of the Union
4,Good Riddance to Chief Wahoo


In [ ]:
frame.shape

(905, 1)

In [ ]:
frame=frame[frame['headline']!='Unknown']

In [ ]:
frame.shape

(821, 1)

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer


In [ ]:
tokenizer=Tokenizer(filters='!"#$%&()*+,-./:;<=>?...',lower=True)

In [ ]:
tokenizer.fit_on_texts(frame.headline.values)

In [ ]:
dict_words=tokenizer.word_index


In [ ]:
#dict_words.values()

dict_values([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 

In [ ]:
def get_sequence_of_tokens(corpus):
    ## tokenization
    total_words = len(tokenizer.word_index) + 1
    
    ## convert data to sequence of tokens 
    input_sequences = []
    for line in corpus:
        token_list = tokenizer.texts_to_sequences([line])[0]
        for i in range(1, len(token_list)):
            n_gram_sequence = token_list[:i+1]
            input_sequences.append(n_gram_sequence)
    return input_sequences, total_words

inp_sequences, total_words = get_sequence_of_tokens(frame.headline.values)
inp_sequences[:10]

[[774, 4],
 [774, 4, 1],
 [774, 4, 1, 236],
 [774, 4, 1, 236, 775],
 [774, 4, 1, 236, 775, 776],
 [774, 4, 1, 236, 775, 776, 90],
 [774, 4, 1, 236, 775, 776, 90, 7],
 [774, 4, 1, 236, 775, 776, 90, 7, 167],
 [774, 4, 1, 236, 775, 776, 90, 7, 167, 37],
 [774, 4, 1, 236, 775, 776, 90, 7, 167, 37, 55]]

In [ ]:
len(inp_sequences)

5124

In [ ]:
total_words

2594

In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
import keras.utils as ku 

In [ ]:
def generate_padded_sequences(input_sequences):
    max_sequence_len = max([len(x) for x in input_sequences])
    input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))
    
    predictors, label = input_sequences[:,:-1],input_sequences[:,-1]
    label = ku.to_categorical(label, num_classes=total_words)
    return predictors, label, max_sequence_len

predictors, label, max_sequence_len = generate_padded_sequences(inp_sequences)

In [ ]:
print(predictors)

[[   0    0    0 ...    0    0  774]
 [   0    0    0 ...    0  774    4]
 [   0    0    0 ...  774    4    1]
 ...
 [   0    0    0 ...  209  538  386]
 [   0    0    0 ...  538  386 2593]
 [   0    0    0 ...  386 2593  532]]


In [ ]:
print(label)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding,Dropout,Dense,LSTM

In [ ]:
def create_model(max_sequence_len, total_words):
    input_len = max_sequence_len - 1
    model = Sequential()
    
    # Add Input Embedding Layer
    model.add(Embedding(total_words, 10, input_length=input_len))
    
    # Add Hidden Layer 1 - LSTM Layer
    model.add(LSTM(100))
    model.add(Dropout(0.1))
    
    # Add Output Layer
    model.add(Dense(total_words, activation='softmax'))

    model.compile(loss='categorical_crossentropy', optimizer='adam')
    
    return model

model = create_model(max_sequence_len, total_words)
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 16, 10)            25940     
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               44400     
_________________________________________________________________
dropout_1 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 2594)              261994    
Total params: 332,334
Trainable params: 332,334
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.fit(predictors, label, epochs=100, verbose=5)

Epoch 1/100
Epoch 2/100
Epoch 3/100
Epoch 4/100
Epoch 5/100
Epoch 6/100
Epoch 7/100
Epoch 8/100
Epoch 9/100
Epoch 10/100
Epoch 11/100
Epoch 12/100
Epoch 13/100
Epoch 14/100
Epoch 15/100
Epoch 16/100
Epoch 17/100
Epoch 18/100
Epoch 19/100
Epoch 20/100
Epoch 21/100
Epoch 22/100
Epoch 23/100
Epoch 24/100
Epoch 25/100
Epoch 26/100
Epoch 27/100
Epoch 28/100
Epoch 29/100
Epoch 30/100
Epoch 31/100
Epoch 32/100
Epoch 33/100
Epoch 34/100
Epoch 35/100
Epoch 36/100
Epoch 37/100
Epoch 38/100
Epoch 39/100
Epoch 40/100
Epoch 41/100
Epoch 42/100
Epoch 43/100
Epoch 44/100
Epoch 45/100
Epoch 46/100
Epoch 47/100
Epoch 48/100
Epoch 49/100
Epoch 50/100
Epoch 51/100
Epoch 52/100
Epoch 53/100
Epoch 54/100
Epoch 55/100
Epoch 56/100
Epoch 57/100
Epoch 58/100
Epoch 59/100
Epoch 60/100
Epoch 61/100
Epoch 62/100
Epoch 63/100
Epoch 64/100
Epoch 65/100
Epoch 66/100
Epoch 67/100
Epoch 68/100
Epoch 69/100
Epoch 70/100
Epoch 71/100
Epoch 72/100
Epoch 73/100
Epoch 74/100
Epoch 75/100
Epoch 76/100
Epoch 77/100
Epoch 78

In [ ]:
def generate_text(seed_text, next_words, model, max_sequence_len):
    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
        predicted = model.predict_classes(token_list, verbose=0)
        
        output_word = ""
        for word,index in tokenizer.word_index.items():
            if index == predicted:
                output_word = word
                break
        seed_text += " "+output_word
    return seed_text.title()

In [ ]:
generate_text('Trump',2,model,max_sequence_len)

'Trump A Way'

In [ ]:
generate_text('India and China',3,model,max_sequence_len)

'India And China War A Shutdown'

In [ ]:
generate_text('President Trump',5,model,max_sequence_len)

'President Trump A Flu World C A Shutdown Of The Shutdown'

In [ ]:
generate_text('South Africa',3,model,max_sequence_len)

'South Africa To The Future'